In [ ]:
import numpy as np 
from citipy import citipy
import gmaps
import requests
import pandas as pd 
from datetime import datetime

from config import weather_api_key
from config_gmaps import g_key

In [ ]:
# Generate 2000 random latitudes and longitudes, and zip them as a list of tuples
lats = np.random.uniform(-90.0,90.0, size=2000)
lngs = np.random.uniform(-180.0,180.0,size=2000)
lats_lngs = zip(lats, lngs)
coordinates = list(lats_lngs)

# Get the nearest city using the citipy module.
cities = list()
for coord in coordinates:
    cities_name = citipy.nearest_city(coord[0],coord[1]).city_name
    if cities_name not in cities:
        cities.append(cities_name)
print(cities[:10], 'Generate', len(cities))

In [ ]:
# Perform an API call with the OpenWeatherMap.

# initial counters for log and sets
record_count = 1 
set_count = 1

city_data = list()
basic_url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&APPID=" + weather_api_key

print('Beginning Data Retrieval     ')
print("-----------------------------")
# use enumerate() method to loop index and item simutanously 
for i, item in enumerate(cities):
    if i % 50 == 0 and i != 0 :
        record_count = 1  
        set_count +=1 
    # build URL for API call
    url = basic_url + '&q=' + item
    # Log the URL, record, and set numbers and the city.
    print(f'Processing Record {record_count} of Set {set_count} | {item}')
    # increment record count
    record_count +=1

    try:
        js = requests.get(url).json()
        city_name = item
        city_country = js['sys']['country']
        city_UTCdate = js['dt']
        city_date = datetime.utcfromtimestamp(city_UTCdate).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = js['coord']['lat']
        city_lon = js['coord']['lon']
        city_max_temp = js['main']['temp_max']
        city_humidity = js['main']['humidity']
        city_cload = js['clouds']['all']
        city_wind_speed = js['wind']['speed']
        city_weather_description = js['weather'][0]['description']

        # raining info
        try:
            js['weather'][0]['main'] == 'Rain'
            city_rain = js['rain']['3h']
        except:
            city_rain = 0.000

        # snowing info
        try:
            js['weather'][0]['main'] == 'Snow'
            city_snow = js['snow']['3h']
        except:
            city_snow = 0    

        # append as a list of dictionaries
        city_data.append({'City':city_name, 'Country':city_country, 'Date':city_date, 
                        'Lat':city_lat, 'Lng': city_lon,'Max Temp':city_max_temp,
                        'Humidity':city_humidity,'Cloudiness': city_cload,'Wind Speed':city_wind_speed,
                        'Current Description':city_weather_description,
                        'Rain inches (last 3 hrs)':city_rain,'Snow inches (last 3 hrs)':city_snow})
    except:
        print("City not found. Skipping...")
        pass
print('-------------------------------')
print('Data Retrieval Complete        ')
print('-------------------------------')

print(len(city_data))